In [1]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Metal device set to: Apple M1


2022-12-21 09:59:29.891321: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-21 09:59:29.891590: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
Total params: 55,744
Trainable params: 55,744
Non-traina

ConvNet이 (image_height, image_width, image_channels) 크기의 입력 텐서를 사용(배치 차원은 포함하지 않음)

채널의 수는 Conv2D 층에 전달된 첫 번째 매개변수에 의해 조절(32개 또는 64개)

완전 연결 네트워크를 마지막에 추가하는데 이때 3D 출력을 1D 텐서로 펼쳐야 함

In [3]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [7]:
# MNIST 이미지에 ConvNet 훈련
from keras import optimizers
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 225

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 225

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5


/Users/hangyojeong/miniforge3/envs/m1_tf_keras/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2022-12-21 10:14:09.582695: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 12s 12ms/step - loss: 0.1732 - accuracy: 0.9454
Epoch 2/5
938/938 [==============================] - 12s 12ms/step - loss: 0.0466 - accuracy: 0.9858
Epoch 3/5
938/938 [==============================] - 11s 12ms/step - loss: 0.0328 - accuracy: 0.9898
Epoch 4/5
938/938 [==============================] - 11s 12ms/step - loss: 0.0246 - accuracy: 0.9922
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 0.0202 - accuracy: 0.9940


In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

 31/313 [=>............................] - ETA: 1s - loss: 0.0432 - accuracy: 0.9839 

2022-12-21 10:15:32.595066: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 2s 6ms/step - loss: 0.0410 - accuracy: 0.9892


0.9892000555992126

Convolution Layer는 입력 특성 맵에서 작은 패치들을 추출하고 이런 모든 패치에 같은 변환을 적용하여 출력 특성 맵을 만듬

출력 특성 맵도 높이와 너비를 가진 3D 텐서

출력 텐서의 깊이는 층의 매개변수로 결정 --> **깊이 축의 채널은 더 이상 RGB 입력처럼 특정 컬러를 의미하는것이 아닌 일종의 필터(filter)를 의미하게 됨**

위의 MNIST 예제의 경우 첫번째 합성곱 층이 (28, 28, 1) 크기의 특성 맵을 입력으로 받아 (26, 26, 32) 크기의 특성 맵을 출력 ==> 이는 입력에 대해 32개의 필터를 적용한다는 의미, 32개의 출력 채널은 각각 26x26 크기의 배열 값을 가짐